<a href="https://colab.research.google.com/github/Mahdi-Saadati/Advanced-Machine-Learning-and-Deep-Neural-Networks/blob/main/11.c.%20Graph%20Neural%20Network%20(GNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<div dir="rtl" align="right">

📌 سناریو:
ما شبکه‌ای از بیماران داریم. هر بیمار چند ویژگی ساده داره:

فشار خون (0=نرمال، 1=بالا)

قند خون (0=نرمال، 1=بالا)

سن (0=جوان، 1=مسن)

بعضی بیماران برچسب‌دار هستن (می‌دونیم بیماری قلبی دارن یا نه) و بقیه بدون برچسب.

هدف: با استفاده از GNN پیش‌بینی کنیم کدام بیماران بیماری قلبی دارند.

In [23]:
import torch                      # کتابخانه اصلی PyTorch
import torch.nn as nn             # تعریف شبکه‌های عصبی
import torch.nn.functional as F   # توابع آماده مثل ReLU و softmax
from torch_geometric.nn import GCNConv  # لایه GCN مخصوص گراف
from torch_geometric.data import Data   # ساختار داده گراف در PyG

# -----------------------------
# 1. تعریف گراف بیماران
# -----------------------------

# ویژگی بیماران (3 ویژگی: فشار خون, قند خون, سن)
# هر سطر = یک بیمار، هر ستون = یک ویژگی
x = torch.tensor([[1.0, 0.0, 1.0],   # بیمار 0: فشار بالا، قند نرمال، مسن
                  [0.0, 1.0, 0.0],   # بیمار 1: فشار نرمال، قند بالا، جوان
                  [1.0, 1.0, 1.0],   # بیمار 2: فشار بالا، قند بالا، مسن
                  [0.0, 0.0, 0.0]],  # بیمار 3: همه نرمال
                 dtype=torch.float)

# روابط بیماران (مثلاً افراد در خانواده یا محیط مشترک)
# edge_index: 2×E (E=تعداد یال‌ها)
# هر ستون = یک یال (from → to)
edge_index = torch.tensor([[0, 1, 2, 3, 0, 2],   # مبدا
                           [1, 0, 3, 2, 2, 0]],  # مقصد
                          dtype=torch.long)

# برچسب بیماری قلبی (0=سالم, 1=بیمار)
y = torch.tensor([1, 0, 1, 0], dtype=torch.long)

# فرض کنید فقط بیمار 0 و 1 داده آموزشی دارند
train_mask = torch.tensor([True, True, False, False])

# ساخت گراف با داده‌ها
data = Data(x=x, edge_index=edge_index, y=y, train_mask=train_mask)

# -----------------------------
# 2. تعریف مدل GCN
# -----------------------------
class GCN(nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        # لایه اول: ورودی=3 ویژگی → 8 نورون مخفی
        self.conv1 = GCNConv(3, 8)
        # لایه دوم: خروجی=2 کلاس (بیمار یا سالم)
        self.conv2 = GCNConv(8, 2)

    def forward(self, x, edge_index):
        # اجرای لایه اول GCN
        x = self.conv1(x, edge_index)
        # اعمال تابع فعال‌سازی ReLU
        x = F.relu(x)
        # اجرای لایه دوم GCN
        x = self.conv2(x, edge_index)
        # تابع softmax برای احتمال هر کلاس
        return F.log_softmax(x, dim=1)

# ساخت مدل
model = GCN()
# تعریف بهینه‌ساز Adam
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# -----------------------------
# 3. آموزش مدل
# -----------------------------
for epoch in range(100):
    model.train()                # حالت آموزش
    optimizer.zero_grad()        # صفر کردن گرادیان‌ها
    out = model(data.x, data.edge_index)  # پیش‌بینی خروجی
    # محاسبه خطا فقط روی گره‌های آموزشی
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()              # محاسبه گرادیان‌ها
    optimizer.step()             # بروزرسانی وزن‌ها
    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss = {loss:.4f}")

# -----------------------------
# 4. تست مدل
# -----------------------------
model.eval()                     # حالت تست
# پیش‌بینی کلاس هر بیمار
pred = model(data.x, data.edge_index).argmax(dim=1)

# چاپ نتایج
print("\nThe actual condition of the patients:", data.y.tolist()) #وضعیت واقعی بیماران
print("Model predicted:", pred.tolist())

Epoch 0, Loss = 0.7064
Epoch 20, Loss = 0.6576
Epoch 40, Loss = 0.6029
Epoch 60, Loss = 0.5286
Epoch 80, Loss = 0.4306

The actual condition of the patients: [1, 0, 1, 0]
Model predicted: [1, 0, 1, 1]


<div dir="rtl" align="right">

📊 تفسیر:

هر بیمار (گره) سه ویژگی داره: فشار خون، قند خون، سن.

مدل با اطلاعات بیماران برچسب‌خورده (بیمار 0 و 1) آموزش می‌بینه.

بعدش برای بیماران ناشناخته (بیمار 2 و 3) پیش‌بینی می‌کنه.

در پروژه واقعی می‌شه ویژگی‌های بیشتر (BMI، کلسترول، سابقه خانوادگی و...) و گراف بزرگ‌تر داشت.